In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAWW', 'AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'PEAK', 'PFG', 'SBNY',
       'SIVB', 'UA', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['AAWW', 'ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VAC', 'VICR', 'VNDA', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW',
       'XEL'],
      dtype='object', length=214)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AAWW', 'ABNB', 'AIRC', 'ALGM', 'ATCO', 'BHF', 'BJ', 'BOH', 'BRBR',
       'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA',
       'DDOG', 'DOW', 'DT', 'DTM', 'ENSG', 'ESAB', 'ETH-USD', 'ETRN', 'FOX',
       'FOXA', 'FRC', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO', 'IAA', 'JBGS', 'JHG',
       'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT', 'OGN', 'OTIS',
       'PDD', 'PEB', 'PGNY', 'RIVN', 'SGOV', 'SHC', 'SITM', 'SJR', 'SWAV',
       'TROX', 'VICI', 'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 303
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 280


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      25
symbols with same volume:   214
symbols with no close:       64

symbols total before drop:                                        1,589
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   280

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,309    1,500
df Close (df_c) after dropped symbols:      1,309    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-05-02  135.020004  137.160004  134.259995  134.520004  1629600.0   
2023-05-03  135.889999  136.539993  134.630005  134.770004  1529100.0   
2023-05-04  135.380005  136.039993  133.759995  133.880005  1103700.0   
2023-05-05  135.300003  135.410004  131.910004  133.300003  1890100.0   
2023-05-08  134.009995  134.210007  131.750000  132.449997  1005300.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-05-02  36.459999  36.720001  35.139999  35.680000  3927200.0  ...   
2023-05-03  35.610001  36.250000  34.860001  35.020000  3564600.0  ...   
2023-05-04  34.660000  35.090000  33.950001  34.380001  3615300.0  ...   
2023-05-05  35.240002  36.349998  34.869999  36.150002  3869400.0  ...   
2023-05-08  37.959999  38.200001  36.689999  36.820000  3749000.0  ...   

                 ZUMZ                                           ZWS  \
                 Open       High    Low      Close    Volume   Open   
Date                                                                  
2023-05-02  16.940001  16.940001  15.87  16.440001  347300.0  21.58   
2023-05-03  16.209999  16.650000  16.17  16.219999  290700.0  21.42   
2023-05-04  16.129999  16.230000  15.70  16.129999  251100.0  21.33   
2023-05-05  16.580000  17.059999  16.43  16.469999  213500.0  21.68   
2023-05-08  16.580000  17.340000  16.43  16.530001  314200.0  21.57   

                                                    
                 High        Low  Close     Volume  
Date                                                
2023-05-02  21.670000  21.059999  21.33  1517700.0  
2023-05-03  21.950001  21.370001  21.52  1063900.0  
2023-05-04  21.480000  21.010000  21.25  1172900.0  
2023-05-05  21.920000  21.200001  21.48  1148000.0  
2023-05-08  21.680000  21.160000  21.48   910000.0  

[5 rows x 6545 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-02,134.520004,35.680000,13.77,124.120003,168.539993,33.029858,36.009998,151.619995,169.160004,42.029999,...,29.639999,143.009995,255.440002,70.080002,42.119999,23.92,27.160000,176.899994,16.440001,21.33
2023-05-03,134.770004,35.020000,13.79,123.800003,167.449997,32.892643,35.959999,149.250000,166.889999,42.049999,...,29.990000,139.619995,258.500000,67.730003,41.470001,22.66,27.389999,177.729996,16.219999,21.52
2023-05-04,133.880005,34.380001,13.51,120.239998,165.789993,32.529999,35.720001,147.360001,166.589996,41.910000,...,29.889999,136.630005,265.000000,66.139999,45.880001,19.93,27.190001,178.470001,16.129999,21.25
2023-05-05,133.300003,36.150002,13.87,123.769997,173.570007,33.790001,35.990002,148.029999,167.990005,42.680000,...,29.860001,137.910004,273.799988,66.360001,47.860001,23.76,27.770000,186.229996,16.469999,21.48
2023-05-08,132.449997,36.820000,14.36,125.750000,173.500000,34.410000,36.139999,147.449997,168.210007,42.320000,...,29.709999,137.440002,277.859985,65.519997,47.380001,24.26,28.299999,184.869995,16.530001,21.48
